In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
from selenium.webdriver.common.by import By
import requests
import ast
import plotly.graph_objects as go
import plotly.express as px
from tqdm import tqdm
from datetime import datetime

In [7]:
driver = webdriver.Chrome()
# Abre la página web
campo = 'auxiliar de contabilidad'
url = 'https://grants.nih.gov/funding/searchguide/index.html#/'
driver.get(url)

In [8]:
input_element = driver.find_element(By.ID, "searchInputBox")
input_element.send_keys("Diet awareness")


select_element = driver.find_element(By.ID, "selectPerPage")
select = Select(select_element)
select.select_by_value("number:100")

In [9]:
driver.find_element(By.CLASS_NAME, "input-group-text.primary").click()

time.sleep(4)

In [10]:
list = {}

contador = 1

while True:
    soup = ''
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(40)
    soup = BeautifulSoup (driver.page_source, 'html.parser')
    
    for row in soup.select('tr.ng-scope'):
        funding = {}
        url = row.select('td a')[0]['href']
        #print(url)
        funding['title'] = row.select('td')[0].text.strip()
        funding['nofo'] = row.select('td')[1].text.strip()
        funding['issuing_organitation'] = row.select('td')[2].text.strip()

        release_date = row.select('td')[3].text.strip()
        if release_date != 'N/A':
            funding['release_date'] = datetime.strptime(release_date, '%b %d, %Y')
        else:
            funding['release_date'] = release_date

        expiration = row.select('td')[4].text.strip()
        if expiration != 'N/A':
            funding['expiration'] = datetime.strptime(expiration, '%b %d, %Y')
        else:
            funding['expiration'] = expiration

        funding['activity_code'] = row.select('td')[5].text.strip()
        list[url] = funding

    #print('Hoja ->', contador ,len(list))
    target_link = soup.find('a', text='Next', attrs={'disabled': 'disabled'})

    contador += 1
    #print('target_link ->', target_link)
    if target_link:
        break
    else:
        elemento = driver.find_element(By.PARTIAL_LINK_TEXT, "Next")
        driver.execute_script("arguments[0].scrollIntoView(true);", elemento)
        time.sleep(5)
        driver.find_element(By.PARTIAL_LINK_TEXT, "Next").click()


C:\Users\c2222\AppData\Local\Temp\ipykernel_1296\2604881484.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  target_link = soup.find('a', text='Next', attrs={'disabled': 'disabled'})


In [11]:
df = pd.DataFrame.from_dict(list, orient='index')
df['url'] = df.index

df = df.reset_index(drop=True)
df.head()
df.to_excel('./listado.xlsx')


In [16]:
df['url']

0      https://grants.nih.gov/grants/guide/notice-fil...
1      https://grants.nih.gov/grants/guide/notice-fil...
2      https://grants.nih.gov/grants/guide/notice-fil...
3      https://grants.nih.gov/grants/guide/notice-fil...
4      https://grants.nih.gov/grants/guide/notice-fil...
                             ...                        
208    https://grants.nih.gov/grants/guide/notice-fil...
209    https://grants.nih.gov/grants/guide/notice-fil...
210    https://grants.nih.gov/grants/guide/notice-fil...
211    https://grants.nih.gov/grants/guide/notice-fil...
212    https://grants.nih.gov/grants/guide/notice-fil...
Name: url, Length: 213, dtype: object

### Recorrido por cada una de las URLS

In [22]:
url = 'https://grants.nih.gov/grants/guide/notice-files/NOT-CA-10-026.html'


response = requests.get(url)
html_content = response.text


html = BeautifulSoup (html_content, 'html.parser')

In [23]:
html

<!-- Google Tag Manager -->
<!-- ADDED ON 05/01/2023 -->
<!-- All services and snippets (AKA tags) have been removed from this file -->
<!-- They are handled in the ralated container in Google Tag Manger -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-TR2CDD');</script>
<!-- End Google Tag Manager -->
<html lang="en-US">
<head>
<title>NOT-CA-10-026: Administrative Supplements to Increase Public Awareness of Pediatric Cancers and Available Treatments and Research</title>
<meta content="NIH Funding Opportunities and Notices in the NIH Guide for Grants and Contracts: Administrative Supplements to Increase Public Awareness of Pediatric Cancers and Available Treatments and Research NOT-CA-10-026. NCI"

In [24]:
information = {}

def extract_paragraphs(text_html,label_init, label_end):
    text_purpose = ''
    inicio = False
    for j in text_html.parent.parent.select('p'):
        if str(j.select('span.Heading3')).find(label_end) != -1:
            inicio = False  
            break
        if str(j).find(label_init) != -1 or inicio == True:
            inicio = True
            text_purpose += str(j)

    text_purpose = text_purpose.replace('<br/>','').replace(f'<p><span class="Heading3">{label_init}</span>','').replace('<strong>','').replace('</strong>','')
    text_purpose = text_purpose.replace('<p>','').replace('</p>','')
    return text_purpose

######################################################################################33333

for i in html.select('span.Heading3'):
    print(i.text)
    if str(i.text).find('Notice Number') != -1:
        information['Notice Number'] = i.parent()[1].text 

    if str(i.text).find('Key Dates') != -1: 
        strong_tags = i.parent.find_all('strong')
        key_dates = {}
        for tag in strong_tags:
            key = tag.text.strip().rstrip(':')
            value = tag.next_sibling.strip() #.find_next('br').next_sibling.strip()
            key_dates[key] = value
        information['dates'] = key_dates 

    if str(i.text).find('Issued by') != -1: 
        information['Issued by'] = i.next_sibling.next_sibling.strip() ### Pendiente validar si en todos los sitios tiene url junto al nombre
    
    if str(i.text).find('Purpose') != -1:
        information['Purpose'] = extract_paragraphs(i.parent.parent,'Purpose','Submission of Requests')

    if str(i.text).find('Submission of Requests') != -1:
        information['Submission of Requests'] = extract_paragraphs(i.parent.parent,'Submission of Requests','Application Procedure')

    if str(i.text).find('Application Procedure') != -1:
        information['Application Procedure'] = extract_paragraphs(i.parent.parent,'Application Procedure','Anticipated  Award Date')

    if str(i.text).find('Anticipated  Award Date') != -1:
        information['Anticipated  Award Date'] = extract_paragraphs(i.parent.parent,'Anticipated  Award Date','Inquiries')

    if str(i.text).find('Inquiries') != -1:
        information['Inquiries'] = extract_paragraphs(i.parent.parent,'Inquiries','--FIN--')

Notice Number: 
Key Dates
Issued by
Purpose
Submission of Requests
Application Procedure
Anticipated  Award Date
Inquiries


In [20]:
for key, value in information.items():
    print(f"{key} = {value}")